In [ ]:
!pip install -U openai-whisper
!pip install webrtcvad
!pip install pydub
!pip install transformers torch
!pip install edge-tts
!pip install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuspa

In [ ]:
import webrtcvad
import numpy as np
from pydub import AudioSegment
import whisper
import torch
from transformers import pipeline
import re
import edge_tts
import asyncio
import nest_asyncio
import time

In [ ]:
WHISPER_MODEL = whisper.load_model(name="base.en")
LAMINI_LLM = pipeline("text2text-generation", model="MBZUAI/LaMini-T5-738M")
vad = webrtcvad.Vad()

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 48.4MiB/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
audio_file_path = "/content/sample_query.m4a"  # Replace with your audio file path
# Specify TTS parameters
output_file = "/content/output_speech.mp3"  # Output file path
voice = 'en-US-MichelleNeural'
# Voice selection
# (Male: 'en-US-GuyNeural', 'en-US-ChristopherNeural', 'en-US-EricNeural', 'en-IN-PrabhatNeural')
# (Female: 'en-US-AriaNeural', 'en-US-JennyNeural', 'en-IN-NeerjaNeural', 'en-US-MichelleNeural')

rate = '+10%'  # Speed adjustment (-100% to +100%)
pitch = '-40Hz'  # Pitch adjustment

In [ ]:
def load_audio(file_path, target_sample_rate=16000):
    '''
    Input : Audio file and target sample rate
    Output : Audio samples in the target channel, sample rate and Normalized in [-1, 1]
    '''
    audio = AudioSegment.from_file(file_path)

    if audio.channels > 1:
        audio = audio.set_channels(1)

    if audio.frame_rate != target_sample_rate:
        audio = audio.set_frame_rate(target_sample_rate)

    audio = audio.normalize(headroom=20.0)

    audio_samples = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0  # Normalize to [-1, 1]

    return audio_samples


def apply_vad(audio_samples, sample_rate=16000, vad_threshold=0.5):
    '''
    Input : Audio samples, sample rate and VAD threshold
    Output : Audio samples after applying VAD
    '''
    vad.set_mode(int(vad_threshold * 3))  # VAD mode ranges from 0 to 3

    frame_duration = 30
    frame_size = int(sample_rate * frame_duration / 1000)
    frames = [audio_samples[i:i + frame_size] for i in range(0, len(audio_samples), frame_size)]

    voiced_frames = []
    for frame in frames:
        if len(frame) == frame_size:
            frame = (frame * 32768).astype(np.int16)
            is_speech = vad.is_speech(frame.tobytes(), sample_rate)
            if is_speech:
                voiced_frames.extend(frame)

    return np.array(voiced_frames).astype(np.float32) / 32768.0  # Convert back to float32 and normalize to [-1, 1]


def transcribe_audio(audio_samples):
    '''
    Input : Audio samples
    Output : Transcription
    '''
    audio_tensor = torch.from_numpy(audio_samples)

    result = WHISPER_MODEL.transcribe(audio_tensor)

    return result['text']


def generate_response(query):
    '''
    Input : Query
    Output : Generated response in 2 lines
    '''
    generated_text = LAMINI_LLM(query, max_length=100, do_sample=True, no_repeat_ngram_size=2)[0]['generated_text']

    first_period = generated_text.find('. ')
    if first_period != -1:
        second_period = generated_text.find('. ', first_period + 1)
        if second_period != -1:
            generated_text = generated_text[:second_period + 1]
        else:
            generated_text = generated_text[:first_period + 1]

    return generated_text.strip()


nest_asyncio.apply()
async def text_to_speech(text, output_file=output_file, voice=voice, rate=rate, pitch=pitch):
    '''
    Input : Text, output file path, voice, rate and pitch
    Output : Audio file
    '''
    tts = edge_tts.Communicate(text, voice=voice, rate=rate, pitch=pitch)

    await tts.save(output_file)


def convert_text_to_speech(text, output_file="output_speech.mp3"):
    '''
    Input : Text and output file path
    Output : Audio file
    '''
    loop = asyncio.get_event_loop()
    loop.run_until_complete(text_to_speech(text, output_file, voice, rate, pitch))

In [ ]:
def main():
    '''
    Main function to execute the entire process
    '''

    start_time = time.time()

    # Load audio
    load_start = time.time()
    audio_samples = load_audio(audio_file_path)
    load_end = time.time()
    print(f"Load audio time: {load_end - load_start:.4f} seconds")

    # Apply VAD
    vad_start = time.time()
    voiced_audio_samples = apply_vad(audio_samples)
    vad_end = time.time()
    print(f"VAD processing time: {vad_end - vad_start:.4f} seconds")

    # Transcribe audio
    transcribe_start = time.time()
    transcription = transcribe_audio(voiced_audio_samples)
    transcribe_end = time.time()
    print(f"Transcription time: {transcribe_end - transcribe_start:.4f} seconds")

    # Generate response
    response_start = time.time()
    response = generate_response(transcription)
    response_end = time.time()
    print(f"LLM Response time: {response_end - response_start:.4f} seconds")

    # Convert text to speech
    tts_start = time.time()
    convert_text_to_speech(response)
    tts_end = time.time()
    print(f"TTS conversion time: {tts_end - tts_start:.4f} seconds")

    total_time = time.time() - start_time
    print(f"Total execution time: {total_time:.4f} seconds")
    print(f'Output file saved at {output_file}')

if __name__ == "__main__":
    main()

Load audio time: 0.3090 seconds
VAD processing time: 0.0111 seconds
Transcription time: 0.1612 seconds
LLM Response time: 6.5657 seconds
TTS conversion time: 0.6973 seconds
Total execution time: 7.7460 seconds
Output file saved at /content/output_speech.mp3


In [ ]:
# Input Audio file
input_audio_file = "/content/sample_query.m4a"
input_audio_segment = AudioSegment.from_file(input_audio_file)

In [ ]:
input_audio_segment

In [ ]:
# LLM response
output_audio_file = "/content/output_speech.mp3"
output_audio_segment = AudioSegment.from_file(output_audio_file)

In [ ]:
output_audio_segment